## Import data and add new variable

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import glob
from matplotlib.ticker import StrMethodFormatter

In [3]:
# list of csv files
files = glob.glob('/content/drive/MyDrive/Projects/Case study 1/trip_data' + "/*.csv")

# joining files with concat and read_csv
df = pd.concat(map(pd.read_csv, files), ignore_index=True)

Add columns for month, day, weekday...

In [4]:
# convert started_at and ended_at to date time
df['date'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])

Add a "ride_length" calculation to all_trips (in minutes)

In [5]:
# add ride_length (in minute)
df['ride_length'] = (df['ended_at'] - df['date']).dt.total_seconds() / 60
df['ride_length'].head()

0    35.366667
1    18.450000
2    17.833333
3    15.033333
4    19.050000
Name: ride_length, dtype: float64

In [6]:
# check if test
df[df['start_station_id'].str.contains('test', case=False, regex=False, na=False)][['start_station_id', 'start_station_name']].drop_duplicates()

,start_station_id,start_station_name
21008,Hubbard Bike-checking (LBS-WH-TEST),Base - 2132 W Hubbard Warehouse
3633414,Pawel Bialowas - Test- PBSC charging station,Pawel Bialowas - Test- PBSC charging station
4578009,DIVVY 001 - Warehouse test station,WEST CHI-WATSON
4589923,Hubbard Bike-checking (LBS-WH-TEST),Base - 2132 W Hubbard
5201347,DIVVY 001 - Warehouse test station,WestChi


In [7]:
bike_df = df[(df['ride_length'] >= 1) & (df['start_lat'] < 44) 
    & (~df['start_station_id'].str.contains('test', case=False, regex=False, na=False)) 
    & (~df['end_station_id'].str.contains('test', case=False, regex=False, na=False))].drop(['ride_id', 'started_at', 'ended_at'], axis=1)
bike_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5799562 entries, 0 to 5900384
Data columns (total 12 columns):
 #   Column              Dtype         
---  ------              -----         
 0   rideable_type       object        
 1   start_station_name  object        
 2   start_station_id    object        
 3   end_station_name    object        
 4   end_station_id      object        
 5   start_lat           float64       
 6   start_lng           float64       
 7   end_lat             float64       
 8   end_lng             float64       
 9   member_casual       object        
 10  date                datetime64[ns]
 11  ride_length         float64       
dtypes: datetime64[ns](1), float64(5), object(6)
memory usage: 575.2+ MB


## Get station info from **GBFS**

In [8]:
station_information = pd.read_json('https://gbfs.divvybikes.com/gbfs/en/station_information.json')
station_information

,data,last_updated,ttl
stations,"[{'capacity': 15, 'eightd_has_key_dispenser': ...",1663317067,5


In [9]:
stations = pd.json_normalize(station_information.data.stations)

In [10]:
stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1419 entries, 0 to 1418
Data columns (total 24 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   capacity                              1419 non-null   int64  
 1   eightd_has_key_dispenser              1419 non-null   bool   
 2   station_id                            1419 non-null   object 
 3   has_kiosk                             1419 non-null   bool   
 4   rental_methods                        681 non-null    object 
 5   lat                                   1419 non-null   float64
 6   external_id                           1419 non-null   object 
 7   name                                  1419 non-null   object 
 8   electric_bike_surcharge_waiver        1419 non-null   bool   
 9   lon                                   1419 non-null   float64
 10  region_id                             676 non-null    object 
 11  eightd_station_se

In [11]:
stations.iloc[0]

capacity                                                                  15
eightd_has_key_dispenser                                               False
station_id                              759171ce-f355-4740-948d-e14bb8f7dc0e
has_kiosk                                                               True
rental_methods                                [KEY, CREDITCARD, TRANSITCARD]
lat                                                                41.651868
external_id                             759171ce-f355-4740-948d-e14bb8f7dc0e
name                                                     Avenue O & 134th St
electric_bike_surcharge_waiver                                         False
lon                                                               -87.539671
region_id                                                                  0
eightd_station_services                                                   []
short_name                                                             20214

In [12]:
stations = stations[['legacy_id', 'short_name', 'name', 'lat', 'lon']]

In [13]:
stations.columns = ['station_id', 'short_name', 'station_name', 'lat', 'lon']

In [14]:
stations.to_csv(r'/content/drive/MyDrive/Projects/Case study 1/station.csv', index=False)

## Update station_id

In [15]:
# check start_station_id not match station_id and short_name
bike_df[(~bike_df['start_station_id'].isin(stations['station_id'])) & (~bike_df['start_station_id'].isin(stations['short_name']))]['start_station_id'].value_counts()

KA1504000152    10297
20246.0          4004
20252.0          2291
20.0             1863
20254.0          1760
                ...  
786                 1
885                 1
811                 1
779                 1
787                 1
Name: start_station_id, Length: 231, dtype: int64

In [16]:
# remove '.0' part in start_stion_id
bike_df['start_station_id'] = bike_df['start_station_id'].str.replace('.0', '', regex=False)

Replace rows with short_name as id to station_id

In [17]:
stations_short_name = stations[stations['short_name'].notnull()]

In [18]:
# replace short_name with id
rows_shortname = bike_df['start_station_id'].isin(stations_short_name['short_name'])
dict = stations_short_name.set_index('short_name')['station_id'].to_dict()
bike_df.loc[rows_shortname, 'start_station_id'] = bike_df.loc[rows_shortname, 'start_station_id'].map(dict)

Rows cannot match by short_name, we use name instead

In [19]:
# check the name not match
bike_df[(~bike_df['start_station_id'].isin(stations['station_id'])) & (~bike_df['start_station_name'].isin(stations['station_name']))]['start_station_name'].value_counts()

Hyde Park Blvd & 55th St          107
Kenton Ave & Palmer St             99
Kostner Ave & Armitage Ave         93
Hyde Park Blvd & 53rd St           75
Midway Orange Line                 66
                                 ... 
Michigan Ave & 96th St              1
Calumet Ave & 103rd St              1
Keeler Ave & 55th St                1
Lake Park Ave & 44th St             1
Newcastle Ave & Wellington Ave      1
Name: start_station_name, Length: 152, dtype: int64

In [20]:
# remove 'Public Rack - ' in name
bike_df['start_station_name'] = bike_df['start_station_name'].str.replace('Public Rack - ', '', regex=False)
stations['station_name'] = stations['station_name'].str.replace('Public Rack - ', '', regex=False)

In [21]:
# replace start_station_id with station_id with match name
rows_name = bike_df['start_station_name'].isin(stations['station_name']) & (~bike_df['start_station_id'].isin(stations['station_id']))
dict = stations.set_index('station_name')['station_id'].to_dict()
bike_df.loc[rows_name, 'start_station_id'] = bike_df.loc[rows_name, 'start_station_name'].map(dict)

In [22]:
fuzzy_name = bike_df['start_station_name'].notnull() & (~bike_df['start_station_id'].isin(stations['station_id']))
bike_df[fuzzy_name]['start_station_name'].value_counts()

Woodlawn Ave & 63rd St - NE               36
Talman Ave & 51st St - midblock           24
Wilton Ave & Diversey Pkwy - Charging     18
Bissell St & Armitage Ave - Charging      17
WEST CHI-WATSON                           16
Kedzie Ave & 61st Pl                      14
Prairie Ave & 47th St - midblock          13
Albany Ave & 63rd St                      12
Pulaski Rd & 41st                          6
Keeler Ave & Madison St                    5
Kedvale Ave & 63rd St                      5
Ashland Ave & 45th St - midblock south     5
Woodlawn Ave & 63rd St - SE                5
DIVVY CASSETTE REPAIR MOBILE STATION       4
NewHastings                                4
Hastings WH 2                              2
Lamon Ave & Archer Ave                     1
Throop/Hastings Mobile Station             1
Name: start_station_name, dtype: int64

In [23]:
!pip install python-Levenshtein
from thefuzz import process
from thefuzz import fuzz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
process.extractOne('Woodlawn Ave & 63rd St - NE', stations['station_name'], scorer=fuzz.token_set_ratio)

('Woodlawn Ave & 63rd St N', 98, 1243)

In [25]:
for i in list(bike_df[fuzzy_name]['start_station_name'].index):
    # get the top 1 closest matches
    match = process.extractOne(bike_df[fuzzy_name]['start_station_name'][i], stations['station_name'], scorer=fuzz.token_set_ratio)

    if match[1] > 90:
        print(bike_df[fuzzy_name]['start_station_name'][i], ' | ', match[0])
        bike_df.loc[i, 'start_station_id'] = stations[stations['station_name'] == match[0]]['station_id'].values

Wilton Ave & Diversey Pkwy - Charging  |  Wilton Ave & Diversey Pkwy
Bissell St & Armitage Ave - Charging  |  Bissell St & Armitage Ave*
Wilton Ave & Diversey Pkwy - Charging  |  Wilton Ave & Diversey Pkwy
Wilton Ave & Diversey Pkwy - Charging  |  Wilton Ave & Diversey Pkwy
Wilton Ave & Diversey Pkwy - Charging  |  Wilton Ave & Diversey Pkwy
Bissell St & Armitage Ave - Charging  |  Bissell St & Armitage Ave*
Wilton Ave & Diversey Pkwy - Charging  |  Wilton Ave & Diversey Pkwy
Bissell St & Armitage Ave - Charging  |  Bissell St & Armitage Ave*
Bissell St & Armitage Ave - Charging  |  Bissell St & Armitage Ave*
Wilton Ave & Diversey Pkwy - Charging  |  Wilton Ave & Diversey Pkwy
Wilton Ave & Diversey Pkwy - Charging  |  Wilton Ave & Diversey Pkwy
Wilton Ave & Diversey Pkwy - Charging  |  Wilton Ave & Diversey Pkwy
Bissell St & Armitage Ave - Charging  |  Bissell St & Armitage Ave*
Wilton Ave & Diversey Pkwy - Charging  |  Wilton Ave & Diversey Pkwy
Bissell St & Armitage Ave - Charging  |

In [26]:
process.extract('Ashland Ave & 45th St - midblock south', stations['station_name'], limit=3, scorer=fuzz.token_set_ratio)

[('Ashland Ave & 45th St', 100, 1000),
 ('Ashland Ave & 45th St  S', 95, 1250),
 ('Ashland Ave & Lake St', 85, 109)]

In [27]:
bike_df.loc[bike_df['start_station_name'] == 'Ashland Ave & 45th St - midblock south', 'start_station_id'] = stations.loc[stations['station_name'] == 'Ashland Ave & 45th St  S', 'station_id'].item()

In [28]:
fuzzy_name = bike_df['start_station_name'].notnull() & (~bike_df['start_station_id'].isin(stations['station_id']))
bike_df[fuzzy_name]['start_station_name'].value_counts()

WEST CHI-WATSON                         16
DIVVY CASSETTE REPAIR MOBILE STATION     4
NewHastings                              4
Hastings WH 2                            2
Throop/Hastings Mobile Station           1
Lamon Ave & Archer Ave                   1
Name: start_station_name, dtype: int64

In [29]:
# drop rows
bike_df.drop(bike_df[fuzzy_name].index, inplace=True)


### Do the same for end_station_id


In [30]:
bike_df['end_station_id'] = bike_df['end_station_id'].str.replace('.0', '', regex=False)

# replace short_name with id
rows_shortname = bike_df['end_station_id'].isin(stations_short_name['short_name'])
dict = stations_short_name.set_index('short_name')['station_id'].to_dict()
bike_df.loc[rows_shortname, 'end_station_id'] = bike_df.loc[rows_shortname, 'end_station_id'].map(dict)

# remove 'Public Rack - ' in name
bike_df['end_station_name'] = bike_df['end_station_name'].str.replace('Public Rack - ', '', regex=False)

# replace start_station_id with station_id with match name
rows_name = bike_df['end_station_name'].isin(stations['station_name']) & (~bike_df['end_station_id'].isin(stations['station_id']))
dict = stations.set_index('station_name')['station_id'].to_dict()
bike_df.loc[rows_name, 'end_station_id'] = bike_df.loc[rows_name, 'end_station_name'].map(dict)

In [31]:
fuzzy_name = bike_df['end_station_name'].notnull() & (~bike_df['end_station_id'].isin(stations['station_id']))
bike_df[fuzzy_name]['end_station_name'].value_counts()

Woodlawn Ave & 63rd St - NE               36
Wilton Ave & Diversey Pkwy - Charging     20
Bissell St & Armitage Ave - Charging      17
Kedzie Ave & 61st Pl                      15
Talman Ave & 51st St - midblock           15
Prairie Ave & 47th St - midblock          13
Albany Ave & 63rd St                       8
WEST CHI-WATSON                            7
DIVVY CASSETTE REPAIR MOBILE STATION       6
Keeler Ave & Madison St                    6
Pulaski Rd & 41st                          6
Kedvale Ave & 63rd St                      5
Ashland Ave & 45th St - midblock south     5
Woodlawn Ave & 63rd St - SE                5
NewHastings                                2
Linder Ave & Archer Ave                    1
Name: end_station_name, dtype: int64

In [32]:
for i in list(bike_df[fuzzy_name]['end_station_name'].index):
    # get the top 1 closest matches
    match = process.extractOne(bike_df[fuzzy_name]['end_station_name'][i], stations['station_name'], scorer=fuzz.token_set_ratio)
    
    if match[1] > 90:
        print(bike_df[fuzzy_name]['end_station_name'][i], ' | ', match[0])
        bike_df.loc[i, 'end_station_id'] = stations[stations['station_name'] == match[0]]['station_id'].values

Wilton Ave & Diversey Pkwy - Charging  |  Wilton Ave & Diversey Pkwy
Bissell St & Armitage Ave - Charging  |  Bissell St & Armitage Ave*
Bissell St & Armitage Ave - Charging  |  Bissell St & Armitage Ave*
Bissell St & Armitage Ave - Charging  |  Bissell St & Armitage Ave*
Bissell St & Armitage Ave - Charging  |  Bissell St & Armitage Ave*
Bissell St & Armitage Ave - Charging  |  Bissell St & Armitage Ave*
Wilton Ave & Diversey Pkwy - Charging  |  Wilton Ave & Diversey Pkwy
Wilton Ave & Diversey Pkwy - Charging  |  Wilton Ave & Diversey Pkwy
Wilton Ave & Diversey Pkwy - Charging  |  Wilton Ave & Diversey Pkwy
Bissell St & Armitage Ave - Charging  |  Bissell St & Armitage Ave*
Bissell St & Armitage Ave - Charging  |  Bissell St & Armitage Ave*
Bissell St & Armitage Ave - Charging  |  Bissell St & Armitage Ave*
Bissell St & Armitage Ave - Charging  |  Bissell St & Armitage Ave*
Bissell St & Armitage Ave - Charging  |  Bissell St & Armitage Ave*
Bissell St & Armitage Ave - Charging  |  Bis

In [33]:
bike_df.loc[bike_df['end_station_name'] == 'Ashland Ave & 45th St - midblock south', 'end_station_id'] = stations.loc[stations['station_name'] == 'Ashland Ave & 45th St  S', 'station_id'].item()

In [34]:
fuzzy_name = bike_df['end_station_name'].notnull() & (~bike_df['end_station_id'].isin(stations['station_id']))
bike_df[fuzzy_name]['end_station_name'].value_counts()

WEST CHI-WATSON                         7
DIVVY CASSETTE REPAIR MOBILE STATION    6
NewHastings                             2
Linder Ave & Archer Ave                 1
Name: end_station_name, dtype: int64

In [35]:
# drop columns
bike_df.drop(bike_df[fuzzy_name].index, inplace=True)

## Filling missing station_id

In [36]:
bike_df.isnull().sum()

rideable_type              0
start_station_name    811138
start_station_id      811135
end_station_name      860418
end_station_id        860418
start_lat                  0
start_lng                  0
end_lat                 5353
end_lng                 5353
member_casual              0
date                       0
ride_length                0
dtype: int64

since start_lat and start_lnd is not null, we will try to see if we can recover start_station_id from them.

In [37]:
bike_df[bike_df['start_station_id'].isnull()].head()

,rideable_type,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,date,ride_length
1180,electric_bike,NaN,NaN,Clark St & Schreiber Ave,453,41.96,-87.69,41.999221,-87.671354,member,2021-07-07 10:17:26,28.066667
1614,electric_bike,NaN,NaN,Lake Shore Dr & Monroe St,76,41.89,-87.63,41.880955,-87.616731,casual,2021-07-18 17:45:04,106.400000
1616,electric_bike,NaN,NaN,Southport Ave & Clybourn Ave,307,41.91,-87.64,41.920755,-87.663708,member,2021-07-24 16:23:31,8.383333
1617,electric_bike,NaN,NaN,Southport Ave & Clybourn Ave,307,41.92,-87.67,41.920690,-87.663757,member,2021-07-01 11:53:56,84.933333
1620,electric_bike,NaN,NaN,Lakefront Trail & Bryn Mawr Ave,760,41.90,-87.69,41.984036,-87.652265,casual,2021-07-04 16:12:43,50.333333


### fill start

In [38]:
stations['100xlat'] = (stations['lat'].round(2)*100).astype(int)

In [39]:
missing_start = bike_df[bike_df['start_station_id'].isnull()].reset_index()[['index', 'start_lat', 'start_lng']]
missing_start['100xlat'] = (missing_start['start_lat']*100).astype(int)
full_start = pd.merge_asof(missing_start.sort_values('start_lng'), stations.sort_values('lon'), left_on='start_lng', right_on='lon', by='100xlat', allow_exact_matches=True, direction='nearest', tolerance=0.01)
full_start[full_start['station_id'].isnull()]

,index,start_lat,start_lng,100xlat,station_id,short_name,station_name,lat,lon
0,1870901,41.94,-87.84,4194,NaN,NaN,NaN,NaN,NaN
1,5699569,41.94,-87.84,4194,NaN,NaN,NaN,NaN,NaN
2,1451668,41.94,-87.84,4194,NaN,NaN,NaN,NaN,NaN
6,1748303,41.94,-87.84,4194,NaN,NaN,NaN,NaN,NaN
7,3335852,41.94,-87.84,4194,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
811130,3618379,41.69,-87.52,4169,NaN,NaN,NaN,NaN,NaN
811131,839587,41.69,-87.52,4169,NaN,NaN,NaN,NaN,NaN
811132,4391094,41.69,-87.52,4169,NaN,NaN,NaN,NaN,NaN
811133,4099423,41.69,-87.52,4169,NaN,NaN,NaN,NaN,NaN


In [40]:
full_start.set_index('index', inplace=True)
bike_df['new_start_station_id'] = bike_df['start_station_id'].fillna(full_start['station_id'])
bike_df.isnull().sum()

rideable_type                0
start_station_name      811138
start_station_id        811135
end_station_name        860418
end_station_id          860418
start_lat                    0
start_lng                    0
end_lat                   5353
end_lng                   5353
member_casual                0
date                         0
ride_length                  0
new_start_station_id      6324
dtype: int64

In [41]:
bike_df[['start_station_id', 'new_start_station_id']]

,start_station_id,new_start_station_id
0,43,43
1,622,622
2,72,72
3,622,622
4,622,622
...,...,...
5900379,245,245
5900380,20,20
5900381,20,20
5900383,51,51


do the same with end_station_id

In [42]:
missing_end = bike_df[bike_df['end_station_id'].isnull() & bike_df['end_lat'].notnull()].reset_index()[['index', 'end_lat', 'end_lng']]
missing_end['100xlat'] = (missing_end['end_lat']*100).astype(int)
full_end = pd.merge_asof(missing_end.sort_values('end_lng'), stations.sort_values('lon'), left_on='end_lng', right_on='lon', by='100xlat', allow_exact_matches=True, direction='nearest', tolerance=0.01)
full_end[full_end['station_id'].isnull()]

,index,end_lat,end_lng,100xlat,station_id,short_name,station_name,lat,lon
0,3316440,41.39,-88.97,4139,NaN,NaN,NaN,NaN,NaN
1,5084304,41.87,-88.14,4187,NaN,NaN,NaN,NaN,NaN
2,3369513,41.90,-87.98,4190,NaN,NaN,NaN,NaN,NaN
3,3333779,41.90,-87.98,4190,NaN,NaN,NaN,NaN,NaN
4,2883764,41.93,-87.96,4193,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
855060,529301,41.68,-87.50,4168,NaN,NaN,NaN,NaN,NaN
855061,593316,41.69,-87.50,4169,NaN,NaN,NaN,NaN,NaN
855062,797295,41.69,-87.50,4169,NaN,NaN,NaN,NaN,NaN
855063,820623,41.68,-87.49,4168,NaN,NaN,NaN,NaN,NaN


In [43]:
full_end.set_index('index', inplace=True)
bike_df['new_end_station_id'] = bike_df['end_station_id'].fillna(full_end['station_id'])
bike_df.isnull().sum()

rideable_type                0
start_station_name      811138
start_station_id        811135
end_station_name        860418
end_station_id          860418
start_lat                    0
start_lng                    0
end_lat                   5353
end_lng                   5353
member_casual                0
date                         0
ride_length                  0
new_start_station_id      6324
new_end_station_id       13512
dtype: int64

In [44]:
bike_df[['end_station_id', 'new_end_station_id']]

,end_station_id,new_end_station_id
0,365,365
1,285,285
2,125,125
3,92,92
4,217,217
...,...,...
5900379,245,245
5900380,20,20
5900381,20,20
5900383,51,51


In [45]:
drop_cols = ['start_station_id', 'start_station_name', 'start_lat', 'start_lng', 'end_station_id', 'end_station_name', 'end_lat', 'end_lng']
bike_df.drop(drop_cols, axis=1, inplace=True)

In [46]:
bike_df.rename(columns={'new_start_station_id':'start_station_id', 'new_end_station_id':'end_station_id'}, inplace=True)

In [47]:
bike_df.isnull().sum()

rideable_type           0
member_casual           0
date                    0
ride_length             0
start_station_id     6324
end_station_id      13512
dtype: int64

In [48]:
bike_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5799518 entries, 0 to 5900384
Data columns (total 6 columns):
 #   Column            Dtype         
---  ------            -----         
 0   rideable_type     object        
 1   member_casual     object        
 2   date              datetime64[ns]
 3   ride_length       float64       
 4   start_station_id  object        
 5   end_station_id    object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 309.7+ MB


## Saving to CSV

In [49]:
bike_df.to_csv(r'/content/drive/MyDrive/Projects/Case study 1/bike_df_map.csv', index=False)